# Grupo Controle vs Grupo Experimental

##### Importação da base de dados

In [1]:
import pandas as pd

# Base de dados
df = pd.read_csv('kc_house_data.csv',delimiter=";")

df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,vendido
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,1180,0,1955,0,98178,475.112,-122.257,1340,5650,1
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,2170,400,1951,1991,98125,47.721,-122.319,1690,7639,1
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,770,0,1933,0,98028,477.379,-122.233,2720,8062,0
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,1050,910,1965,0,98136,475.208,-122.393,1360,5000,0
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,1680,0,1987,0,98074,476.168,-122.045,1800,7503,1


##### Análise da base

* Observado o volume de vendas por CEP (zipcode)
* Filtrado as maiores vendas e observado a correlação entre o números de quartos com as maiores vendas

In [2]:

# RESET INDEX evita o nivelamento do agrupamento, facilitando o acesso aos campos

# Soma de vendas por cep e quartos
df_grouped = df.groupby(["zipcode","bedrooms"]).agg(sum_vendido=('vendido','sum')).reset_index()

# Maiores vendas por cep
df_grouped_2 = df_grouped.groupby(["zipcode"]).agg(max_vendido=('sum_vendido','max')).reset_index()

df_merged = pd.merge(df_grouped, df_grouped_2, on=['zipcode'])

# Filtro com apenas as maiores vendas por CEP e QUARTOS
df_filtered = df_merged[df_merged['sum_vendido'] == df_merged['max_vendido']]

# Exibe o resumo por QUARTOS
display(df_filtered.groupby('bedrooms').count())


,zipcode,sum_vendido,max_vendido
bedrooms,,,
3,68,68,68
4,2,2,2


##### Análise dos imóveis não vendidos

Objetivo verificar o motivo de não terem sido vendidos

Continuaremos explorando a correlação com a variavel QUARTOS

In [3]:
# Contagem por número de QUARTOS dos imóveis NÃO vendidos

print(df[df['vendido']==0].groupby('bedrooms')['id'].count())

bedrooms
0        5
1       93
2     1218
4     3156
5      724
6      117
7       16
8        8
9        3
10       2
11       1
33       1
Name: id, dtype: int64


### Experimento

No publico não vendido separar 10% de imoveis, este será o grupo experimental, o grupo controle será o restante

No grupo experimental acrescentar 10% de imoveis com 3 quartos


In [4]:
df_nao_vendidos = df[df['vendido']==0]

# RANDOM_STATE é o SEED da aleatoriedade, fazendo com que consigamos obter a mesma massa de dados
# em uma outra execução
df_experimento = df_nao_vendidos.sample(frac=0.1, random_state=30)

df_exp_3_quartos = df_experimento.sample(frac=0.1, random_state=30)

df_exp_3_quartos['ajuste_quartos'] = 3

df_experimento_merged = pd.merge(df_experimento,df_exp_3_quartos,on=['id'],how='left')

condicao = df_experimento_merged['ajuste_quartos'] == 3.0

df_experimento_merged.loc[condicao, 'bedrooms_x']=df_experimento_merged['ajuste_quartos']

colunas = []
colunas.append(df.columns.to_list()[0])
for col in range(1, len(df.columns.to_list())):
    colunas.append(f'{df.columns.to_list()[col]}_x')

colunas.append('ajuste_quartos')

df_experimento_merged_selected = df_experimento_merged[colunas]

df_experimento_merged_selected.to_csv('experimento.csv')

In [5]:
# df_experimento_merged_selected.head()

df_experimento_aplicado = pd.read_csv('experimento - ajustado.csv',delimiter=";")

# df_experimento_aplicado.head()

df_experimento_aplicado_grouped = df_experimento_aplicado.groupby(["zipcode_x","bedrooms_x"]).agg(sum_vendido=('vendido_experimento','sum')).reset_index()

# Maiores vendas por cep
df_experimento_aplicado_grouped_2 = df_experimento_aplicado_grouped.groupby(["zipcode_x"]).agg(max_vendido=('sum_vendido','max')).reset_index()

df_experimento_aplicado_merged = pd.merge(df_experimento_aplicado_grouped, df_experimento_aplicado_grouped_2, on=['zipcode_x'])

# Filtro com apenas as maiores vendas por CEP e QUARTOS
df_experimento_aplicado_filtered = df_experimento_aplicado_merged[df_experimento_aplicado_merged['sum_vendido'] == df_experimento_aplicado_merged['max_vendido']]

# Exibe o resumo por QUARTOS
display(df_experimento_aplicado_filtered.groupby('bedrooms_x')['zipcode_x'].count())


bedrooms_x
2    15
3     7
4    50
5     8
6     1
Name: zipcode_x, dtype: int64

## Resumo:

O experimento mostrou que NÃO é apenas o fato de ter 3 comodos que as casas vendem mais